# import and functions 

In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


def find_sync_file(mouse_file, animal_dat_path):
    ## find sync file: 
    for item in os.listdir(animal_dat_path):
        if mouse_file.split('_')[0] in item:
            if mouse_file.split('_')[1] == item[-1]:
                for folder in os.listdir(animal_dat_path + item):
                    if folder.split('_')[0].split('g')[-1] == mouse_file.split('_')[-1]:
                        current_animals_video_path = os.path.join(animal_dat_path + item,folder)+ r"\\video\tracking\\"
                        current_animals_behav_sync_path = os.path.join(animal_dat_path + item,folder)+ r"\\behav_sync\\"
                        for stage in os.listdir(current_animals_behav_sync_path):
                            if 'postsleep' in stage or 'post_sleep' in stage:
                                sync_path = current_animals_behav_sync_path + stage +r"\\"

    if mouse_file == '148_2_2':
        sync_df = pd.read_csv(sync_path + r'Postsleep_Ephys_Camera_sync.csv', encoding='cp1252')
    else:
        sync_df = pd.read_csv(sync_path + r'Postsleep_Ephys_Camera_sync.csv')
        
    return sync_df 

def extract_number(filename):
    """Extracts the numerical part from the filename."""
    match = re.search(r'channel-(\d+)', filename)
    if match:
        return int(match.group(1))
    return float('inf')  # Return infinity if no number is found#

# new way based on Kime t al Ganguly
def butter_bandpass_filter(data, lowcut, highcut, fs, order_hp, order_lp):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist

    # High-pass filtering
    b_hp, a_hp = butter(order_hp, low, btype='high', analog=False)
    filtered_hp = filtfilt(b_hp, a_hp, data)

    # Low-pass filtering
    b_lp, a_lp = butter(order_lp, high, btype='low', analog=False)
    filtered_band = filtfilt(b_lp, a_lp, filtered_hp)

    return filtered_band


# main

taken from: https://www.nature.com/articles/nature12112

 To examine SWRs, the local field potential was band-pass filtered between 150 and 250 Hz, and the absolute value of the Hilbert transform of this filtered signal was then smoothed (Gaussian kernel, s.d. = 12.5 ms). This processed signal was averaged across all tetrodes and ripple events were identified as local peaks with an amplitude greater than 3 s.d. above the mean, using only periods when the rat’s velocity was less than 5 cm s−1. The start and end boundaries for each event were defined as the point when the signal crossed the mean.

 I will do the same:

 - bandpass between 150 and 250Hz
 - absolute value of hilbert transform and smooth (guassian 12.5ms)
 - average this. 
 - peaks greater than 3.sd are ripples - start and end = mean crossing points

In [ ]:
# load in the raw LFP data:
### secondary processing of the LFP, to create actual LFP averages. 
LFP_dat_path = r"Z:\projects\sequence_squad\revision_data\emmett_revisions\oscillations\hippocampus_lfp\extracted_LFP\\"
animal_dat_path = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

# check which mice havent been processed yet 
for mouse in os.listdir(LFP_dat_path):
    temp_path = os.path.join(LFP_dat_path,mouse)
    if not 'processed_data' in os.listdir(temp_path):
        print(mouse)

seq006_1_1
seq006_1_2
seq006_1_3
seq006_1_4
seq006_1_5
seq006_1_6
seq007_1_1
seq007_1_2
seq007_1_3


In [ ]:
mouse_file = 'seq006_1_1'

mouse_path = os.path.join(LFP_dat_path, mouse_file)
data_channels = []
channel_names = []
for channels in os.listdir(mouse_path):
    if 'striatum' in channels:
        print(channels)
        channel_path = os.path.join(mouse_path, channels)
        try:
            data_channels += [np.load(channel_path)]
        except:
            data_channels += [np.load(channel_path, allow_pickle=True)]
        channel_names +=[channels]
    elif 'timestamps' in channels:
        timestamps_df = pd.read_csv(os.path.join(mouse_path, channels))
   
# reorder them so that they ar ein channel order
sorted_inds = np.argsort([extract_number(item) for item in channel_names])
channel_names =  np.array(channel_names)[sorted_inds]
data_channels = np.array(data_channels)[sorted_inds]    


# hack to make sure that timestamps are correct length
if not len(data_channels[0]) == len(timestamps_df):
    timestamps_df = timestamps_df[0:len(data_channels[0])]
    print('timestamps and data channels are not the same length, truncating timestamps to match data channels')
            
### chop the data down to just the post sleep 

# load in the sync df to get the start time of the sleep period
sync_df = find_sync_file(mouse_file, animal_dat_path)
sleep_period_ephys_start_time = sync_df.Camera_time_Ephys_Aligned[0]
# make a mask for the sleep period
chopping_mask = timestamps_df.ephys_timestamp > sleep_period_ephys_start_time
# apply that mask to each channel and the timestamps
sleep_period_timestamps_df = timestamps_df[chopping_mask]
sleep_period_data_channels = []
for i,dat in enumerate(data_channels):
    sleep_period_data_channels += [dat[chopping_mask]]

In [ ]:
## chose which channels to keep 
fig, axs = plt.subplots(len(sleep_period_data_channels), 2, figsize=(7, 4))  # 2 rows, 1 column
for i,dat in enumerate(sleep_period_data_channels):
    #downsample just for visualisation
    dat = dat[::10]
    axs[i][0].plot(dat)
    axs[i][1].plot(dat[10000:10100])
    axs[i][0].set_title(f"{i} | {channel_names[i]}", fontsize=6, color = 'red')

choose non noisy channels and bandpass between 150-250Hz

In [ ]:
### choose channels 
chosen_channels_bool = [1,1,1,1,1,1,1,1]

channel_ = []
for ind,dat in enumerate(sleep_period_data_channels):
    if chosen_channels_bool[ind] == 1:
        channel_ += [dat]
lfp_all = np.vstack(channel_)     


### !!! this isnt going to work....i fucking low passed the data at 30Hz... so i need to re-extract the hippocampal LFP data from the raw data...


#bandpass filter the data
fs = 2500  # Sampling frequency
lowcut = 150  # Cutoff frequency for high-pass filter (Hz)
highcut = 250   # Cutoff frequency for low-pass filter (Hz)
order_hp = 2  # Order of the high-pass Butterworth filter
order_lp = 2  # Order of the low-pass Butterworth filter
delta_filtered_z = butter_bandpass_filter(average_lfp_z, lowcut, highcut, fs, order_hp, order_lp) 



NameError: name 'sleep_period_data_channels' is not defined

compute the hilbert transform for each channel 